In [1]:
# Data Acquisition

In [20]:
import numpy as np
import pandas as pd
import time
import ast

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import regex as re


%config InlineBackend.figure_format = 'retina'
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Facebook Log In - Mobile Browser

In [ ]:
#log in details
user = 'kitsamho@gmail.com'
password = 'Da3FNUqN'
brand = 'CostaCoffee'

# user = input("User Email: ")
# password = input("Password : ")
# brand = input("What brand do you want to scrape: ")

#initialise web driver
driver = webdriver.Chrome('./chromedriver')

#FACEBOOK LOG IN - First Screen (There are two variants of this page so try/except loop covers all instances)
driver.get('https://mobile.facebook.com/')
email = driver.find_element_by_id('m_login_email')
email.send_keys(user)
try:
    pass_1 = driver.find_element_by_id('m_login_password')
    pass_1.send_keys(password)
except:
    pass_2 = driver.find_element_by_xpath('//*[@id="login_form"]/ul/li[2]/div/input')
    pass_2.send_keys(password)
try:
    click_1 = driver.find_element_by_id('u_0_5')
    click_1.click()
except:  
    click_2= driver.find_element_by_xpath('//*[@id="login_form"]/ul/li[3]/input')
    click_2.click()

#FACEBOOK LOG IN - Second Screen (This is always called)
password_keep = driver.find_element_by_class_name('_4g34')
password_keep.click()

#GO TO BRAND PAGE    
driver.get(f'https://mobile.facebook.com/{brand}')
#driver.get(f'https://mobile.facebook.com/story.php?story_fbid={fbid}&id={i}')    
           
#FACEBOOK LOG IN - Brand Page (For some reason it expects you to log in again so we have to do the process again) 
click_brand = driver.find_element_by_xpath('//*[@id="mobile_login_bar"]/div[2]/a[1]')
click_brand.click() 

#flat log on or brand log on
#flat log on
try:
    click_third = driver.find_element_by_xpath('//*[@id="u_0_3"]')
    click_third.click()
    
    pass_3 = driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/form/div[1]/div/div/input')
    pass_3.send_keys(password)
    
    log_on = driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/form/div[2]/button')
    log_on.click()
#brand log on
except:
    
    f = driver.find_element_by_xpath('//*[@id="m_login_password"]')
    f.send_keys(password) 
    
    g = driver.find_element_by_xpath('//*[@id="u_0_5"]')
    g.click()

start = time.time()
while time.time()-start<10:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#SCRAPING STARTS HERE

#list of each posts text/content
post_text = []
#list of each posts comments and share count
post_comments_shares = []
#list of dates when each post was made
post_date = []
#list of total interactions(e.g. all likes, love, haha, wow, sad, angry)
post_all_response = []
#row labels for brand scraped
brand_list = []

#THIS IS WHERE WE INITATE THE SCRAPE  
item = driver.find_elements_by_class_name('_3drp')

for each in item:
    try:
        post_text.append(each.find_element_by_tag_name('p').text)
    except:
        post_text.append(np.nan)
    try:
        post_date.append(each.find_element_by_tag_name('abbr').text)
    except:
        post_date.append(np.nan)
    try:
        post_comments_shares.append(each.find_element_by_class_name('_1fnt').text)
    except:
        post_comments_shares.append(np.nan)
    try:
        post_all_response.append(each.find_element_by_class_name('_1g06').text)
    except:
        post_all_response.append(np.nan)
        
count = len(post_text)
while count > 0:
    brand_list.append(brand)
    count -= 1

items = pd.DataFrame({'Post_Content': post_text,
                      #'Type': post_type,
                      'Date': post_date,
                      'Comments_Shares': post_comments_shares,
                      'All_Responses': post_all_response,
                      'Brand' : brand_list
                     })

print(f"Facebook scrape of {brand} scraped {len(post_text)} Facebook posts.")
    

## Cleaning

In [311]:
#several formatting and tidying

items.Comments_Shares = items.Comments_Shares.str.replace('Comments',',')
items.Comments_Shares = items.Comments_Shares.str.replace('Shares','')
items.Comments_Shares = items.Comments_Shares.str.replace(' ','')
items[['Comments','Shares']] = items.Comments_Shares.str.split(',', expand = True)
items.drop(['Comments_Shares'],axis=1,inplace=True)

#remove 'K', convert to integers, 
def kformat(x):
    try:
        if 'K' in x:
            return float(x.replace('K',''))*1000
        else:
            return float(x)
    except:
        return x
    
items = items.applymap(kformat)

items.All_Responses = items.All_Responses.apply(lambda x: int(x))  
items.Comments = items.Comments.apply(lambda x: int(x)) 
items.Shares = items.Shares.apply(lambda x: int(x))  

df = items[['Brand','Post_Content','All_Responses','Comments','Shares']]


In [313]:
df

,Brand,Post_Content,All_Responses,Comments,Shares
0,Tesco,"There’s beef, there’s cauliflower wellington, ...",810,368,48
1,Tesco,"All year round, our Community Food Connection ...",187,70,69
2,Tesco,Heard about our Halloumi fries? Grab all the c...,177,122,13
3,Tesco,"White, red, or bubbles? Just don’t forget the ...",826,273,85
4,Tesco,"Food that’s better for you, your wallet and th...",263,159,52
5,Tesco,"Pudding. Or is it afters? For 22% of us, it's ...",930,234,79
6,Tesco,Hear the story of how one mum just changed nap...,10000,1600,3800
7,Tesco,Are you one of the 64% sitting down to turkey ...,147,173,19
8,Tesco,We know that everyone's Christmas is unique. W...,423,450,151
9,Tesco,NaN,255,160,12


## Final Data Frame

In [ ]:
#initialise web driver
driver = webdriver.Chrome('./chromedriver')

#FACEBOOK LOG IN - First Screen (There are two variants of this page so try/except loop covers all instances)
driver.get('https://mobile.facebook.com/')
email = driver.find_element_by_id('m_login_email')
email.send_keys(user)
try:
    pass_1 = driver.find_element_by_id('m_login_password')
    pass_1.send_keys(password)
except:
    pass_2 = driver.find_element_by_xpath('//*[@id="login_form"]/ul/li[2]/div/input')
    pass_2.send_keys(password)
try:
    click_1 = driver.find_element_by_id('u_0_5')
    click_1.click()
except:  
    click_2= driver.find_element_by_xpath('//*[@id="login_form"]/ul/li[3]/input')
    click_2.click()

#FACEBOOK LOG IN - Second Screen (This is always called)
password_keep = driver.find_element_by_class_name('_4g34')
password_keep.click()

#GO TO BRAND PAGE    
driver.get(f'https://mobile.facebook.com/{brand}')
#driver.get(f'https://mobile.facebook.com/story.php?story_fbid={fbid}&id={i}')    
           
#FACEBOOK LOG IN - Brand Page (For some reason it expects you to log in again so we have to do the process again) 
click_brand = driver.find_element_by_xpath('//*[@id="mobile_login_bar"]/div[2]/a[1]')
click_brand.click() 

#flat log on or brand log on
#flat log on
try:
    click_third = driver.find_element_by_xpath('//*[@id="u_0_3"]')
    click_third.click()
    
    pass_3 = driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/form/div[1]/div/div/input')
    pass_3.send_keys(password)
    
    log_on = driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/form/div[2]/button')
    log_on.click()
#brand log on
except:
    
    f = driver.find_element_by_xpath('//*[@id="m_login_password"]')
    f.send_keys(password) 
    
    g = driver.find_element_by_xpath('//*[@id="u_0_5"]')
    g.click()

start = time.time()
while time.time()-start<5:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#THIS IS WHERE WE INITATE THE SCRAPE    
item = driver.find_elements_by_class_name('_3drp')

#list of each posts text/content
post_text = []

#list of each posts comments and share count
post_comments_shares = []

#list of dates when each post was made
post_date = []

#list of total interactions(e.g. all likes, love, haha, wow, sad, angry)
post_all_response = []

#detailed responses
post_like = []
post_love = []
post_wow = []
post_angry = []
post_sad = []
post_haha = []

post_check = []

ids_ = []
fbids_ = []


for each in item:
    info = ast.literal_eval(each.find_element_by_class_name('_56be').get_attribute("data-ft"))
    fbid_ = info['mf_story_key']
    id_ = info['page_id']
    ids_.append(id_)
    fbids_.append(fbid_)
    try:
        post_text.append(each.find_element_by_tag_name('p').text)
    except:
        post_text.append(np.nan)
    try:
        post_date.append(each.find_element_by_tag_name('abbr').text)
    except:
        post_date.append(np.nan)
    try:
        post_comments_shares.append(each.find_element_by_class_name('_1fnt').text)
    except:
        post_comments_shares.append(np.nan)
    try:
        post_all_response.append(each.find_element_by_class_name('_1g06').text)
    except:
        post_all_response.append(np.nan)
        
for i, fbid in zip(ids_, fbids_):
    driver.get(f'https://mobile.facebook.com/story.php?story_fbid={fbid}&id={i}')    
    url = (f'https://mobile.facebook.com/story.php?story_fbid={fbid}&id={i}')
  
    #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     try:
#         post_check.append(driver.find_element_by_class_name('msg').text)
#     except:
#         post_check.append(np.nan)
    
    driver.get(f'https://mobile.facebook.com/ufi/reaction/profile/browser/?ft_ent_identifier={fbid}') 
   
    try:
        post_like.append(driver.find_element_by_id('_10tn').text)
    except:
        post_like.append(0)
    try:
        post_love.append(driver.find_element_by_id('u_0_18').text)
    except:
        post_love.append(0)
    try:
        post_wow.append(driver.find_element_by_id('u_0_19').text)
    except:
        post_wow.append(0)
    try:
        post_angry.append(driver.find_element_by_id('u_0_20').text)
    except:
        post_angry.append(0)
    try:
        post_sad.append(driver.find_element_by_id('u_0_21').text)
    except:
        post_sad.append(0)
    try:
        post_haha.append(driver.find_element_by_id('u_0_22').text)
    except:
        post_haha.append(0)

In [ ]:
#checking consistency with numbers in field

print("post text: ",len(post_text))
print("post_comments_shares : ", len(post_comments_shares))
print("post_date :",len(post_date))
print("post_all_response : ",len(post_all_response))
print("post_like :",len(post_like))
print("post_love :",len(post_love))
print("post_angry :",len(post_angry))
print("post_sad :",len(post_sad))
print("post_haha :",len(post_haha))
print("post_check :",len(post_check))
print("ids_ :",len(ids_))
print("fbids_ :",len(fbids_))



In [224]:
items = pd.DataFrame({'Post_Content': post_text,
                      #'Type': post_type,
                      'Date': post_date,
                      'Comments_Shares': post_comments_shares,
                      'All_Responses': post_all_response,
                      'Likes': post_like,
                      'Love': post_love,
                      'haha': post_haha,
                      'Wow': post_wow,
                      'Angry': post_angry,
                      'Sad': post_sad
        
                     })